## 0. Read the Documentation to have a better understanding of the data.

1. There are six data resources, including `films`, `people`, `planets`,  `species`, `starships` and `vehicles`. We could use different tables to store these data.  
2. Responses are in Json format, where we could get attributes of the `films`, `people` etc.

In [3]:
# install the required modules
! pip install requests pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
# import the required modules
import sqlite3
import requests
import pandas as pd

## 1. Define the data schema for tables.

In [19]:
# define the data schmea and write relevant SQL statements according to the documentation
create_tables = ['''
CREATE TABLE IF NOT EXISTS people (
    name TEXT PRIMARY KEY,
    birth_year TEXT,
    eye_color TEXT,
    gender TEXT,
    hair_color TEXT,
    height TEXT,
    mass TEXT,
    skin_color TEXT,
    homeworld TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS films (
    title TEXT PRIMARY KEY,
    episode_id INTEGER,
    opening_crawl TEXT,
    director TEXT,
    producer TEXT,
    release_date DATE,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS starships (
    name TEXT PRIMARY KEY,
    model TEXT,
    starship_class TEXT,
    manufacturer TEXT,
    cost_in_credits TEXT,
    length TEXT,
    crew TEXT,
    passengers TEXT,
    max_atmosphering_speed TEXT,
    hyperdrive_rating TEXT,
    MGLT TEXT,
    cargo_capacity TEXT,
    consumables TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS vehicles (
    name TEXT PRIMARY KEY,
    model TEXT,
    vehicle_class TEXT,
    manufacturer TEXT,
    length TEXT,
    cost_in_credits TEXT,
    crew TEXT,
    passengers TEXT,
    max_atmosphering_speed TEXT,
    cargo_capacity TEXT,
    consumables TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS species (
    name TEXT PRIMARY KEY,
    average_height TEXT,
    average_lifespan TEXT,
    classification TEXT,
    designation TEXT,
    eye_colors TEXT,
    hair_colors TEXT,
    homeworld TEXT,
    language TEXT,
    skin_colors TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS planets (
    name TEXT PRIMARY KEY,
    diameter TEXT,
    rotation_period TEXT,
    orbital_period TEXT,
    gravity TEXT,
    population TEXT,
    climate TEXT,
    terrain TEXT,
    surface_water TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
'''
]

In [21]:
# connect to the database
conn = sqlite3.connect('swapi.db')
c = conn.cursor()

# create relevant tables
for sql_statement in create_tables:
    c.execute(sql_statement)

conn.commit()

## 2. Get the data from the Star Wars API and store them in local databases.

### 2.1 Using People as the First Example

In [28]:
# check the requests output
people_response = requests.get('https://swapi.dev/api/people/')
people_data = people_response.json()
print(people_data['results'])

[{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}, {'name': 'C-3PO', 'height': '167', 'mass': '75', 'hair_color': 'n/a', 'skin_color': 'gold', 'eye_color': 'yellow', 'birth_year': '112BBY', 'gender': 'n/a', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/

The result is a list of jsons(persons), we could sort them and store them one by one.

```
{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.dev/api/planets/1/', 'films': ['https://swapi.dev/api/films/1/', 'https://swapi.dev/api/films/2/', 'https://swapi.dev/api/films/3/', 'https://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['https://swapi.dev/api/vehicles/14/', 'https://swapi.dev/api/vehicles/30/'], 'starships': ['https://swapi.dev/api/starships/12/', 'https://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.dev/api/people/1/'}, 
```

In [30]:
def save_to_person(person):
    '''
    Insert individual person data to the people table.
    '''
    with sqlite3.connect('swapi.db') as conn:
        c = conn.cursor()
        c.execute("""
            INSERT INTO people (name, height, mass, hair_color, skin_color, eye_color, birth_year, gender, homeworld, url, created, edited)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (person['name'], person['height'], person['mass'], person['hair_color'], person['skin_color'], person['eye_color'], person['birth_year'], person['gender'], person['homeworld'], person['url'], person['created'], person['edited']))
        conn.commit()


In [31]:
# iterate through the json response and store them into the table
for person in people_data['results']:
    save_to_person(person)

In [35]:
# check if we have already store it correctly
c = conn.cursor()
check_data = c.execute("SELECT * FROM people LIMIT 5").fetchall()
print(check_data)
conn.commit()

[('Luke Skywalker', '19BBY', 'blue', 'male', 'blond', '172', '77', 'fair', 'https://swapi.dev/api/planets/1/', 'https://swapi.dev/api/people/1/', '2014-12-09T13:50:51.644000Z', '2014-12-20T21:17:56.891000Z'), ('C-3PO', '112BBY', 'yellow', 'n/a', 'n/a', '167', '75', 'gold', 'https://swapi.dev/api/planets/1/', 'https://swapi.dev/api/people/2/', '2014-12-10T15:10:51.357000Z', '2014-12-20T21:17:50.309000Z'), ('R2-D2', '33BBY', 'red', 'n/a', 'n/a', '96', '32', 'white, blue', 'https://swapi.dev/api/planets/8/', 'https://swapi.dev/api/people/3/', '2014-12-10T15:11:50.376000Z', '2014-12-20T21:17:50.311000Z'), ('Darth Vader', '41.9BBY', 'yellow', 'male', 'none', '202', '136', 'white', 'https://swapi.dev/api/planets/1/', 'https://swapi.dev/api/people/4/', '2014-12-10T15:18:20.704000Z', '2014-12-20T21:17:50.313000Z'), ('Leia Organa', '19BBY', 'brown', 'female', 'brown', '150', '49', 'light', 'https://swapi.dev/api/planets/2/', 'https://swapi.dev/api/people/5/', '2014-12-10T15:20:09.791000Z', '201

### 2.2 Extract the Responses and Store the Data to Other Tables.

In [ ]:
# check the requests output
remain_tables = ['films', 'starships', 'vehicles', 'species', 'planets']
people_response = requests.get('https://swapi.dev/api/people/')
people_data = people_response.json()
print(people_data['results'])